In [ ]:
import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')

In [ ]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
import numpy as np
import pandas as pd
import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential  
from keras.layers import Dense  
from keras.layers import LSTM  
from keras.layers import Dropout
from sklearn.metrics import mean_absolute_error
from math import sqrt
from sklearn.metrics import mean_squared_error

In [ ]:
file_path = "/home/bharat/proj/time_series1/4200_C005_2019_03_03.tsv"
df = pd.read_csv(file_path, sep=',', header=None)
df.columns = ["kunag", "matnr", "date", "quantity","price"]

In [ ]:
def n_series(input_df,kunag,matnr):
    input_df = input_df[input_df["quantity"] >=0]    
    df = input_df.copy()
    n_df1 = df[(df["kunag"] == kunag) & (df["matnr"] == matnr)]
    n_df1.date = n_df1.date.apply(lambda x : pd.to_datetime(x,format = '%Y%m%d', errors='ignore'))
    n_df1 = n_df1.sort_values('date')
    n_df1.set_index('date',inplace=True)
    weekly_resampled_data = n_df1.quantity.resample('W').sum() 
    weekly_resampled_data = weekly_resampled_data.replace(np.nan, 0)
    individual_series = weekly_resampled_data.to_frame()
    individual_series = individual_series.reset_index()
    return individual_series


def train_test_split(input_df,kunag,matnr,i):
    data= n_series(input_df,kunag,matnr)
    train = data[0:-i]    
    test = data[-i:]
    return train,test

In [ ]:
train,test=  train_test_split(df,500056565,100278,16)

In [ ]:
dataframe = n_series(df,500056565, 100278)

In [ ]:
order = 16
test_points = 16
df_testing_complete = pd.read_csv(r'testing.csv')
test_predictions = []
print(dataframe[102:])
for k in range(0, test_points):
    df = dataframe[:-test_points + k]
    df_training_complete = df
    print(df_training_complete.shape)
  
    df_training_processed = df_training_complete.iloc[:, 1:2].values  

    scaler = MinMaxScaler(feature_range = (0, 1))
    df_training_scaled = scaler.fit_transform(df_training_processed)  

    features_set = []  
    labels = []  
    for i in range(order , df.shape[0] - 1):  
        features_set.append(df_training_scaled[i-order:i, 0])
        labels.append(df_training_scaled[i, 0])
    
    print("X", features_set[-1])
    features_set, labels = np.array(features_set), np.array(labels)  
    features_set = np.reshape(features_set, (features_set.shape[0], features_set.shape[1], 1))  
    print(features_set.size)
    model = Sequential()  
    model.add(LSTM(units=50, return_sequences=True, input_shape=(features_set.shape[1], 1)))  
    model.add(Dropout(0.2))  
    model.add(LSTM(units=50, return_sequences=True))  
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=True))  
    model.add(Dropout(0.2))
    model.add(LSTM(units=50))  
    model.add(Dropout(0.2))
    model.add(Dense(units = 1))  
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')  
    
    model.fit(features_set, labels, epochs = 5, batch_size = 32)  
    df_testing_processed = df_testing_complete.iloc[k:k+1, 1:2].values 
    df_total = pd.concat((df_training_complete['quantity'], df_testing_complete['quantity']), axis=0)  
    test_inputs = df_total[len(df_total) - len(df_testing_complete) - order + k:].values  
    test_inputs = test_inputs.reshape(-1,1)  
    test_inputs = scaler.transform(test_inputs) 

    test_features = []   
    test_features.append(test_inputs[0:order, 0])
    
    test_features = np.array(test_features)  
    test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], 1))
    print("Y", test_features[-1][-1][-1])
    predictions = model.predict(test_features)  
    predictions = scaler.inverse_transform(predictions)  
    test_predictions.append(predictions)


In [ ]:
test_predictions_1 = [i[0][0] for i in test_predictions]

In [ ]:
test_predictions_1

In [ ]:
dataframe[-5:]

In [ ]:
# predictions

In [ ]:
train,test=  train_test_split(df,500056565,100278,16)

In [ ]:
df_c = pd.DataFrame(data=test_predictions_1)

In [ ]:
test.reset_index(inplace = True)
pred = pd.concat([test, df_c], axis=1, join_axes=[test.index])
pred.set_index("date",inplace = True)
del pred["index"]
del pred["quantity"]
del test["index"]


In [ ]:

mae = mean_absolute_error(test.quantity, pred[0])
rms = sqrt(mean_squared_error(test.quantity, pred[0]))
print("mae :",mae)
print("rms :",rms)

In [ ]:
# dataframe.tail()

In [ ]:
test.tail()

In [ ]:
train.head()

In [ ]:
dataframe.set_index('date',inplace=True)
train.set_index('date',inplace=True)
test.set_index('date',inplace=True)

In [ ]:
plt.figure(figsize=(16,8))
plt.plot( dataframe, marker='.', color='blue')
plt.plot( train, marker='.', color='blue')
plt.plot( test, marker='.', color='orange')
plt.plot(pred,marker = ".",color = 'green')
plt.xlabel("time")
plt.ylabel('quantity')
# plt.savefig("/home/rahul/Downloads/bharat/time_series1/high_freq_plots/" + 'Graph_{}.png'.format(index), format="PNG")  

plt.show()


In [ ]:

# model = Sequential()  
# model.add(LSTM(units=50, return_sequences=True, input_shape=(features_set.shape[1], 1)))  
# model.add(Dropout(0.2))  
# model.add(LSTM(units=50, return_sequences=True))  
# model.add(Dropout(0.2))
# model.add(LSTM(units=50, return_sequences=True))  
# model.add(Dropout(0.2))
# model.add(LSTM(units=50))  
# model.add(Dropout(0.2))
# model.add(Dense(units = 1))  
# model.compile(optimizer = 'adam', loss = 'mean_squared_error')  
# model.fit(features_set, labels, epochs = 100, batch_size = 32)  
# df_testing_complete = pd.read_csv(r'testing.csv')  
# df_testing_processed = df_testing_complete.iloc[:, 1:2].values 

In [ ]:
# df_total = pd.concat((df_training_complete['quantity'], df_testing_complete['quantity']), axis=0)  
# test_inputs = df_total[len(df_total) - len(df_testing_complete) - 16:].values  
# test_inputs = test_inputs.reshape(-1,1)  
# test_inputs = scaler.transform(test_inputs) 

# test_features = []  
# for i in range(16, 16 + 1):  
#     test_features.append(test_inputs[i-16:i, 0])
    
# test_features = np.array(test_features)  
# test_features = np.reshape(test_features, (test_features.shape[0], test_features.shape[1], 1))

# predictions = model.predict(test_features)  
# predictions = scaler.inverse_transform(predictions)  